In [3]:
!pip install lightgbm
!pip install tqdm
!pip install gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import gensim.models
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 24.2MB 65.7MB/s ta 0:00:011▎                            | 2.5MB 81.7MB/s eta 0:00:0101   34% |███████████                     | 8.3MB 83.6MB/s eta 0:00:01███████████████▌           | 15.5MB 81.1MB/s eta 0:00:01�████      | 19.6MB 100.6MB/s eta 0:00:01�██▋ | 23.1MB 97.3MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 69.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.4MB 58.7MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 10.7MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 56.3MB/s ta 0:00:01
    100% |████████████████████████████████| 6.4MB 93.3MB/s ta 0:00:011  8% |██▊                      

In [8]:
train = pd.read_csv('train_set.csv', sep='\t')
test = pd.read_csv('test.csv', sep='\t')

In [9]:
train

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...
5,9,3819 4525 1129 6725 6485 2109 3800 5264 1006 4...
6,3,307 4780 6811 1580 7539 5886 5486 3433 6644 58...
7,10,26 4270 1866 5977 3523 3764 4464 3659 4853 517...
8,12,2708 2218 5915 4559 886 1241 4819 314 4261 166...
9,3,3654 531 1348 29 4553 6722 1474 5099 7541 307 ...


In [10]:
texts=pd.concat([train['text'],test['text']]).reset_index(drop=True)
texts

0         2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1         4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2         7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3         7159 948 4866 2109 5520 2490 211 3956 5520 549...
4         3646 3055 3055 2490 4659 6065 3370 5814 2465 5...
5         3819 4525 1129 6725 6485 2109 3800 5264 1006 4...
6         307 4780 6811 1580 7539 5886 5486 3433 6644 58...
7         26 4270 1866 5977 3523 3764 4464 3659 4853 517...
8         2708 2218 5915 4559 886 1241 4819 314 4261 166...
9         3654 531 1348 29 4553 6722 1474 5099 7541 307 ...
10        3659 3659 1903 1866 4326 4744 7239 3479 4261 4...
11        6469 1066 1623 1018 3694 4089 3809 4516 6656 3...
12        3772 4269 3433 6122 2035 4531 465 6565 498 358...
13        4630 2210 1641 1854 1641 4543 3017 4409 5430 6...
14        1103 6835 3433 2107 5165 7543 3598 5229 1946 2...
15        4412 5988 5036 4216 7539 5644 1906 2380 2252 6...
16        2400 4411 5968 2612 6920 4464 

In [ ]:
# from collections import Counter
# all_lines = ' '.join(list(texts))
# word_count = Counter(all_lines.split(" "))
# word_count = sorted(word_count.items(), key=lambda d:d[1], reverse = True)
# print(len(word_count))
# print(word_count[0])
# print(word_count[-1])


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1),max_features=3000)
data_set= vectorizer.fit_transform(texts).toarray()
 

In [12]:
train_feat=data_set[:200000]
test_feat=data_set[200000:]

In [13]:
import lightgbm as lgb
from sklearn.metrics import f1_score
x_train,x_test,y_train,y_test=train_test_split(train_feat,train['label'],test_size=0.2,random_state=1)

clf=lgb.LGBMClassifier(
    n_estimators=700,
    boosting_type='gbdt',
    learning_rate=0.1,
    objective='multiclass',
    num_class=14)

clf.fit(x_train,y_train,
         eval_set=[(x_train,y_train),(x_test,y_test)],
         verbose=100,
         early_stopping_rounds=100)

y_pred=clf.predict(x_test)
label=clf.predict(test_feat)
score=f1_score(y_test, y_pred, average='macro')
print("score:",score)

label=pd.DataFrame(label,columns=['label'])
label.to_csv("lgb_label.csv",index=False)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.162213	valid_1's multi_logloss: 0.236798
[200]	training's multi_logloss: 0.0771227	valid_1's multi_logloss: 0.193839
[300]	training's multi_logloss: 0.0444026	valid_1's multi_logloss: 0.184226
[400]	training's multi_logloss: 0.0274827	valid_1's multi_logloss: 0.182016
Early stopping, best iteration is:
[393]	training's multi_logloss: 0.0283997	valid_1's multi_logloss: 0.182004
score: 0.9366422724717209


In [ ]:
===========================================================

In [6]:
word2vec_model = gensim.models.Word2Vec(texts.values, size=63, iter=10,window=12, min_count=1,
                                        workers=8,seed=2020,sample=1e-5,sg=1,hs=1)
word2vec_model.save('texts.model')
print("success!!!!!!!!!")

success!!!!!!!!!


In [8]:
train['text']
def Get_vector(trace):    
    word2vec_model = gensim.models.Word2Vec.load('texts.model')
    index2word_set = set(word2vec_model.wv.index2word)
    startTime = time.time()
    vector_list = []
    num=0
    for cutWords in trace:
        i=0
        article_vector = np.zeros((word2vec_model.trainables.layer1_size))
        for cutword in cutWords:
            if cutword in index2word_set:
                article_vector = np.add(article_vector, word2vec_model.wv[cutword])
                i += 1
        cutWord_vector = np.divide(article_vector, i)
        vector_list.append(list(cutWord_vector))
        del cutWord_vector
        del article_vector
        num+=1
        if num %50000==0:
            print("前",num,"个耗时：",(time.time() - startTime))
    return pd.DataFrame(vector_list)
train_vector=Get_vector(train['text'].values)
test_vector=Get_vector(test['text'].values)


前 50000 个耗时： 1665.9355807304382
前 100000 个耗时： 3205.9941194057465
前 150000 个耗时： 4838.847198486328
前 200000 个耗时： 6518.758146047592
前 50000 个耗时： 1672.5965094566345


In [9]:
train_vector.to_csv("train_vec.csv",index=False)
test_vector.to_csv("test_vector.csv",index=False)

In [13]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

word_vectorizer.fit(texts)
train_word_features = word_vectorizer.transform(train['text'].values)
test_word_features = word_vectorizer.transform(test['text'].values)




<200000x6977 sparse matrix of type '<class 'numpy.float64'>'
	with 56074040 stored elements in Compressed Sparse Row format>

In [16]:
x_train,x_test,y_train,y_test=train_test_split(train_word_features,train['label'],test_size=0.2,random_state=1)
clf = LogisticRegression(C=4)
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
train_scores = clf.score(x_train, y_train)
print(train_scores, f1_score(y_pred, y_test, average='macro'))


D:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9484625 0.9218392965754442


In [17]:
label=clf.predict(test_word_features)
label=pd.DataFrame(label,columns=['label'])
label.to_csv("label.csv",index=False)

In [18]:
import lightgbm as lgb
from sklearn.metrics import f1_score
x_train,x_test,y_train,y_test=train_test_split(train_word_features,train['label'],test_size=0.2,random_state=1)

clf=lgb.LGBMClassifier(
    n_estimators=700,
    boosting_type='gbdt',
    learning_rate=0.1,
    objective='multiclass',
    num_class=14)

clf.fit(x_train,y_train,
         eval_set=[(x_train,y_train),(x_test,y_test)],
         verbose=100,
         early_stopping_rounds=100)

y_pred=clf.predict(x_test)
label=clf.predict(test_vector)
score=f1_score(y_test, y_pred, average='macro')
print("score:",score)

label=pd.DataFrame(label,columns=['label'])
label.to_csv("lgb_label.csv",index=False)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.754206	valid_1's multi_logloss: 1.09841
Early stopping, best iteration is:
[57]	training's multi_logloss: 0.241205	valid_1's multi_logloss: 0.320826


ValueError: Number of features of the model must match the input. Model n_features_ is 6977 and input n_features is 63 